In [1]:
ls

drive/  sample_data/


In [2]:
%cd drive/My Drive/Colab Notebooks/rag_nvidia

/content/drive/My Drive/Colab Notebooks/rag_nvidia


In [3]:
ls

rag_gen_hyde.ipynb  requirements.txt  Untitled0.ipynb


In [4]:
!pip install -r requirements.txt

In [5]:
ls

rag_gen_hyde.ipynb  requirements.txt  Untitled0.ipynb


In [6]:

import streamlit as st
import os
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain_community.document_loaders import WebBaseLoader
from langchain.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader
import time

In [10]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API keys
nvidia_api_key = os.getenv('NVIDIA_API_KEY')


# Ensure the API keys are loaded correctly
if nvidia_api_key is None:
    raise ValueError("API keys not found. Please check your .env file.")

print("nvidia_api_found")

nvidia_api_found


In [13]:
# Document ingestion and vector store creation
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Initialize embeddings and loader
embeddings = NVIDIAEmbeddings()
loader = PyPDFDirectoryLoader("./US_Census")

# Load documents
docs = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
final_documents = text_splitter.split_documents(docs[:30])

# Create FAISS vector store
vectors = FAISS.from_documents(final_documents, embeddings)


In [14]:
# Define the prompt and LLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

# Initialize the LLM
llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

# Define the prompt template
prompt_template = """
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question.
<context>
{context}
<context>
Questions:{input}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)


In [15]:
# Implement the retrieval chain
import time

def answer_question(query):
    # Create the document chain
    document_chain = create_stuff_documents_chain(llm, prompt)
    # Get the retriever
    retriever = vectors.as_retriever()
    # Create the retrieval chain
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    # Measure the response time
    start = time.process_time()
    response = retrieval_chain.invoke({'input': query})
    end_time = time.process_time() - start

    return response, end_time

In [17]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 82.3 MB/s 

In [18]:
# Create Gradio Interface
import gradio as gr

def generate_answer(query):
    response, response_time = answer_question(query)
    answer = response['answer']
    context_docs = "\n\n".join([doc.page_content for doc in response["context"]])
    return f"Answer:\n{answer}\n\nResponse Time: {response_time} seconds\n\nContext Documents:\n{context_docs}"

iface = gr.Interface(
    fn=generate_answer,
    inputs="text",
    outputs="text",
    title="Nvidia NIM Demo",
    description="Ask questions about the documents"
)

iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8c522ff1b07e214904.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [19]:
# Create Gradio Interface
import gradio as gr


# Define Gradio Interface functions
def embed_documents():
    global vectors
    vectors = FAISS.from_documents(final_documents, embeddings)
    return "Vector Store DB Is Ready"

def generate_answer(query):
    response, response_time = answer_question(query)
    answer = response['answer']
    context_docs = "\n\n".join([doc.page_content for doc in response["context"]])
    return f"Answer:\n{answer}\n\nResponse Time: {response_time} seconds\n\nContext Documents:\n{context_docs}"

# Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# Nvidia NIM Demo")
    query_input = gr.Textbox(label="Enter Your Question From Documents")
    embed_button = gr.Button("Documents Embedding")
    output_text = gr.Textbox(label="Output", interactive=False)

    embed_button.click(embed_documents, inputs=None, outputs=output_text)
    query_input.submit(generate_answer, inputs=query_input, outputs=output_text)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d56604b66df81b8dfc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
